##### Loading packages

In [1]:
from pymongo import MongoClient
import json
import time
from bson.json_util import dumps
from elasticsearch import Elasticsearch
import pandas as pd
from py2neo import Graph, Node, Relationship
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import treetaggerwrapper
import warnings
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
from time import time, ctime
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from scipy import spatial
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
start = time()

##### Connection to MongoDB

In [3]:
#Server
server = MongoClient('')
#BD
db = server['aura_pmi_dl']
#Collection
collection = db['document_profiles']

##### Connection to Neo4j

In [4]:
graph = Graph("bolt://159.84.108.111:7687", auth=("neo4j", ""))

##### Connection to Elasticsearch

In [5]:
es = Elasticsearch("")

##### Loading Stopwords

In [10]:
#FR
stpw_fr = list(set(stopwords.words('french')))

In [11]:
#EN
stpw_en = list(set(stopwords.words('english')))

##### Lemmatization

In [12]:
tagger_fr = treetaggerwrapper.TreeTagger(TAGLANG='fr')

In [13]:
tagger_en = treetaggerwrapper.TreeTagger(TAGLANG='en')

##### Data in french

In [14]:
##### DATA IN FRENCH


#First query to check number of hits
query = {"query": {"match_all": {}}}
res = es.search(index="document_index_fr", body=query, scroll='2m',)
result_size = res['hits']['total']['value']
print("Got %d Hits:" % result_size)
# Get the scroll ID
first = True
sid = res['_scroll_id']
scroll_size = len(res['hits']['hits'])

liste_text_fr = []
liste_id_fr = []    
while ((scroll_size > 0) | first):   
    
    for hit in res['hits']['hits']:
        
        identifier = hit['_source']['identifier']
        text_fr = hit['_source']['content']
        liste_id_fr.append(identifier)
    
    
        #PROCESSING THE RESULT
        
        #Stopwords removal
        word_tokens = word_tokenize(text_fr, language='french') 
        filtered_text = [w for w in word_tokens if w not in stpw_fr] 
        filtered_text =[term.lower() for term in filtered_text if term.isalpha()]
        filtered_text_fr = " ".join(filtered_text)
       
        #Lemmatization
       
        text = filtered_text_fr.lower()
        tags = tagger_fr.tag_text(text)
        liste_lemmas = [tag.split('\t')[2] for tag in tags]
        lemmatized_text_fr = " ".join(liste_lemmas)
        liste_text_fr.append(lemmatized_text_fr)    
    
    ##Query again
    res = es.scroll(scroll_id=sid, scroll='2m')
    # Update the scroll ID
    first = False
    sid = res['_scroll_id']
    scroll_size = len(res['hits']['hits'])
    
print("END")
    

Got 8649 Hits:
END


##### Data in english

In [ ]:
##### DATA IN ENGLISH


#First query to check number of hits
query = {"query": {"match_all": {}}}
res = es.search(index="document_index_en", body=query, scroll='2m',)
result_size = res['hits']['total']['value']
print("Got %d Hits:" % result_size)
# Get the scroll ID
first = True
sid = res['_scroll_id']
scroll_size = len(res['hits']['hits'])

liste_text_en = []
liste_id_en = []    
while ((scroll_size > 0) | first):   
    
    for hit in res['hits']['hits']:
        
        identifier = hit['_source']['identifier']
        text_en = hit['_source']['content']
        liste_id_en.append(identifier)
    
    
        #PROCESSING THE RESULT
        
        #Stopwords removal
        word_tokens = word_tokenize(text_en, language='english') 
        filtered_text = [w for w in word_tokens if w not in stpw_en] 
        filtered_text =[term.lower() for term in filtered_text if term.isalpha()]
        filtered_text_en = " ".join(filtered_text)
       
        #Lemmatization
       
        text = filtered_text_en.lower()
        tags = tagger_en.tag_text(text)
        liste_lemmas = [tag.split('\t')[2] for tag in tags]
        lemmatized_text_en = " ".join(liste_lemmas)
        liste_text_en.append(lemmatized_text_en)    
    
    ##Query again
    res = es.scroll(scroll_id=sid, scroll='2m')
    # Update the scroll ID
    first = False
    sid = res['_scroll_id']
    scroll_size = len(res['hits']['hits'])
    
print("END")
    

Got 41324 Hits:


##### Vectorization - Embedding

In [16]:
#Global lists
liste_id = liste_id_fr + liste_id_en
liste_text = liste_text_fr + liste_text_en
del liste_text_en
del liste_id_en
del liste_text_fr
del liste_id_fr

NameError: name 'liste_text_en' is not defined

In [26]:
NB_COMPONENTS = 100
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(liste_text)]
model = Doc2Vec(documents, vector_size=NB_COMPONENTS, window=3, min_count=1, workers=4)

In [18]:
fname = get_tmpfile("my_doc2vec_model")

In [19]:
model.save(fname)

In [24]:
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [21]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

##### Creation of representations in Neo4j

In [ ]:
#####Creation of representation nodes
for identifier in liste_id:
    graph.run('CREATE (c:Document:Refined {identifier:"'+ identifier +'", vocabulary:"global_vocabulary", format:"embedding"}) RETURN c')

In [ ]:
##### Linking representation nodes with main documents
query = '''MATCH (d:Object),(p:Refined {vocabulary:"global_vocabulary", format:"embedding"}) WHERE d.identifier = p.identifier
            CREATE (d)-[r:REPRESENTATION]->(p)
            RETURN d,p'''
res = graph.run(query)

In [ ]:
##### Generation of similarities

#scc_similarity_matrix = spearmanr(liste_doc_vectors, liste_doc_vectors)
#cos_similarity_matrix = cosine_similarity(liste_doc_vectors, liste_doc_vectors)

##### Creation of representation in MongoDB

In [ ]:
##### Adding the "representations field"
for i, identifier in enumerate(liste_id):
    identifier = liste_id[i]
    dimensions = np.array(['dim'+str(i) for i in range(NB_COMPONENTS)])
    doc_vocab = pd.DataFrame({"dim":dimensions, "value":model.docvecs[i]}) 
    doc_vocab.index = doc_vocab["dim"]
    representation = doc_vocab.to_dict(orient='records') #Generation of the representation
    query = {'_id': identifier}
    new_values = {'$addToSet': {'representations': {"vocabulary":"global_vocabulary", "format":"embedding", "data":representation} }}
    collection.update_one(query, new_values)

##### Creation of Similarity Links

array([-0.1273403 , -0.14608477,  0.4224325 ,  0.04118891,  0.76641405,
       -0.17481771,  1.2649163 , -1.1716287 ,  0.90775836, -0.8898628 ,
        0.5827632 , -1.5164006 , -0.07379433,  0.4499898 ,  0.07833008,
       -0.48350087,  0.68391824, -0.21687557, -0.20227014, -1.0840961 ,
        0.67306376, -0.24656947, -0.60194266,  0.3168104 , -1.2110342 ,
       -0.35495126, -0.561662  , -0.5264341 , -0.65603584,  0.84768075,
        0.625523  ,  0.54525894,  0.85951424, -0.61103547, -0.79932   ,
       -0.32540563, -0.1675622 , -1.2314767 , -0.15207194, -0.21316959,
       -1.1078626 , -0.716714  , -0.8542841 , -0.18060179, -0.93470615,
        0.15863393, -1.647496  ,  0.979502  , -0.12137932, -0.29938632,
        0.9153041 , -0.71219724, -0.33108902,  0.8375322 , -0.05632454,
        0.29518408,  0.16091841,  0.1025452 ,  0.26846913, -0.81526554,
       -0.43847927,  0.08741318,  0.04842928,  0.93069994, -0.26138   ,
        0.909917  , -0.7010395 ,  0.19297259,  0.3426578 ,  0.08

In [33]:
#infer_vector = model.infer_vector(liste_text[0].split())

In [41]:
#model = Load_model('model.doc2vec')

#infer_vector = model.infer_vector(s)

#similar_documents = model.docvecs.most_similar([model.docvecs[0]], topn = 5)


In [42]:
#similar_documents

[(0, 0.9999998807907104),
 (8331, 0.7180832624435425),
 (128, 0.6914499998092651),
 (2746, 0.6743021011352539),
 (1188, 0.6647195816040039)]

In [45]:
#similar_documents[0][1]

0.9999998807907104

1
2


In [ ]:
nb = 5
seuil=0.60
k=0
l=0
for i, identifier1 in enumerate(liste_id):
    similar_documents = model.docvecs.most_similar([model.docvecs[i]], topn = nb+1)
    for j in range(1, nb+1):
        num = similar_documents[j][0]
        sim = similar_documents[j][1]
        identifier2 = liste_id[num]
        if(sim >= seuil):#We only consider similarities higher than a certain threshold
            query = '''MATCH (r1:Refined {vocabulary:"global_vocabulary", identifier:"'''+ identifier1 +'''", format:"embedding"}),
            (r2:Refined {vocabulary:"global_vocabulary", identifier:"'''+ identifier2 +'''", format:"embedding"})
                CREATE (r1)<-[r:SIMILARITY {measure: "cosine_similarity", value:''' + str(sim) + '''}]-(r2)
                RETURN r1,r2'''
            graph.run(query)
            l +=1
    if i % 1000 == 0:
        print("i = ", str(i) ," time: ", ctime(time()))  

##### Indexing

In [ ]:
try:
    resp = collection.create_index({"representations.vocabulary":-1})
    print("Index created on vocabulary")
except:
    print("unable to create index on vocabulary in MongoDB")

In [ ]:
try:
    resp = collection.create_index([("representations.vocabulary",1),("representations.format",1)])
    print("Index created on format")
except:
    print("unable to create index on format in MongoDB")

In [ ]:
try:
    resp = collection.create_index([("representations.data.term",1)])
    print("Index created on term")
except:
    print("unable to create index on term in MongoDB")

In [ ]:
try:
    resp = collection.create_index([("representations.data.dim",1)])
    print("Index created on dim")
except:
    print("unable to create index on dim in MongoDB")

In [ ]:
del liste_id
del liste_text

In [ ]:
done = time()
elapsed = done - start
print("*"*20)
print("STARTED : "+ctime(start) )
print("END : "+ctime(done) )
print("TIME ELAPSED:" + str(elapsed/60) + " MINUTES" )